### 🛠️ 도구 오류 처리 (Tool Error Handling)
LangChain 에이전트에서 도구 실행 중 발생하는 오류를 **사용자 지정 방식으로 처리**하려면 **미들웨어**(Middleware)를 사용합니다.</br>
이는 에이전트의 핵심 로직을 변경하지 않고도 오류 발생 시 모델에게 피드백을 제공하여, 에이전트가 실패를 인식하고 다음 행동을 결정할 수 있도록 합니다.

#### 1. 오류 처리 방식: `@wrap_tool_call` 데코레이터

이 데코레이터는 도구 호출을 감싸는 함수를 정의하게 해줍니다.</br>
함수 내부에서는 try...except 블록을 사용하여 도구 실행 중 발생할 수 있는 Exception을 포착합니다.

#### 2. 사용자 지정 오류 메시지 반환

오류가 발생했을 때, 에이전트는 일반적인 Python 예외를 반환하는 대신 사용자 지정 오류 메시지를 담은 ToolMessage를 모델에게 반환해야 합니다.</br>

+ 목적: 모델(LLM)은 이 ToolMessage의 내용을 관찰(Observation)하고, 추론(Reasoning) 단계에서 오류가 발생했음을 인지하고 사용자에게 적절한 피드백을 제공하거나 다른 도구 호출을 시도하도록 유도할 수 있습니다.

+ 필수 요소: 반환되는 ToolMessage에는 오류의 원인(예: Tool error: ...)과 함께, 어떤 도구 호출에 대한 응답인지 식별하기 위한 **tool_call_id**가 반드시 포함되어야 합니다.

In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [11]:
from langchain.tools import tool
from langchain.agents import create_agent

@tool
def search(query: str) -> str:
    """정보를 검색합니다."""
    # ! 의도적으로 오류를 발생시키는 조건 추가
    if "오류 발생" in query:
        raise ValueError("실습을 위한 의도적인 오류입니다.")
        
    return f"검색 결과: {query}"

@tool
def get_weather(location: str) -> str:
    """특정 장소의 날씨 정보를 가져옵니다."""
    return f"날씨 정보: {location}은(는) 맑고 22°C입니다."

In [14]:
from langchain.agents import create_agent
from langchain.agents.middleware import wrap_tool_call
from langchain_core.messages import ToolMessage


@wrap_tool_call
def handle_tool_errors(request, handler):
    """도구 실행 오류를 사용자 지정 메시지로 처리합니다."""
    try:
        return handler(request)
    except Exception as e:
        # 모델에게 사용자 지정 오류 메시지를 반환합니다.
        # 주의: LangGraph 상태 스키마에 ToolMessage를 반환할 때 'tool_call_id'는 필수입니다.
        return ToolMessage(
            content=f"도구 오류: 입력을 확인하고 다시 시도해 주세요. ({str(e)})",
            tool_call_id=request.tool_call["id"] # tool_call_id를 명시적으로 추가해야 정상 작동합니다.
        )

# 미들웨어를 에이전트에 적용
agent = create_agent(
    model="google_genai:gemini-2.5-flash",
    tools=[search, get_weather],
    middleware=[handle_tool_errors]
)

In [15]:
agent.invoke(
    {"messages": [{"role": "user", "content": "검색 도구를 사용해서 '오류 발생'이라는 키워드를 검색해 주세요."}]}
)

{'messages': [HumanMessage(content="검색 도구를 사용해서 '오류 발생'이라는 키워드를 검색해 주세요.", additional_kwargs={}, response_metadata={}, id='67ee9b55-6b20-44a2-a817-d7850f533631'),
  AIMessage(content='', additional_kwargs={'function_call': {'name': 'search', 'arguments': '{"query": "\\uc624\\ub958 \\ubc1c\\uc0dd"}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': [], 'grounding_metadata': {}, 'model_provider': 'google_genai'}, id='lc_run--bd25aa90-fa2d-4d38-a3ea-564f6b6dd7eb-0', tool_calls=[{'name': 'search', 'args': {'query': '오류 발생'}, 'id': '0af85394-d9a5-4ee7-b5ca-9476dc9adfd5', 'type': 'tool_call'}], usage_metadata={'input_tokens': 99, 'output_tokens': 48, 'total_tokens': 147, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 33}}),
  ToolMessage(content='도구 오류: 입력을 확인하고 다시 시도해 주세요. (실습을 위한 의도적인 오류입니다.)', id='29c69843-63d6-4ec1-817a-c1a870a8cc20', tool_call_i